In [ ]:
#Imports and gpu check
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
from keras.callbacks import EarlyStopping
from keras import Sequential
from keras.layers import Dense, Dropout, Conv1D, Reshape, MaxPool1D, Flatten
from keras.initializers import HeNormal
from keras.regularizers import l1_l2
!nvidia-smi

Thu Nov 19 19:35:22 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.38       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#Downloading the data
!wget http://phoenix.mit.bme.hu:49080/kgt/MIGT_2020_traindata_V1.tsv

--2020-11-19 19:35:22--  http://phoenix.mit.bme.hu:49080/kgt/MIGT_2020_traindata_V1.tsv
Resolving phoenix.mit.bme.hu (phoenix.mit.bme.hu)... 152.66.253.41
Connecting to phoenix.mit.bme.hu (phoenix.mit.bme.hu)|152.66.253.41|:49080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2301643 (2.2M) [text/tab-separated-values]
Saving to: ‘MIGT_2020_traindata_V1.tsv.7’

MIGT_2020_traindata 100%[===================>]   2.19M  1.80MB/s    in 1.2s    

2020-11-19 19:35:24 (1.80 MB/s) - ‘MIGT_2020_traindata_V1.tsv.7’ saved [2301643/2301643]



In [ ]:
#Reading the input with pandas and numpy
table = pd.read_csv("MIGT_2020_traindata_V1.tsv", delimiter='\t')
nparr=table.to_numpy()

In [ ]:
#Data preprocessing
outChapter=nparr[:, 0:1].reshape(3200).astype(float)
outSectionFloat=nparr[:, 1:2].reshape(3200).astype(float)
outIsExtra=nparr[:, 2:3].reshape(3200).astype(float)
input=nparr[:, 3:4].reshape(3200)

In [ ]:
#Pretrained model import from tensorflow hub.
#This model's job was to process the textual paragraphs of the input.
#After this layer, I used 3 different models to predict the 3 different outputs.

importedmodel= "https://tfhub.dev/google/tf2-preview/nnlm-en-dim128/1"

hub_layer = hub.KerasLayer(importedmodel, output_shape=[128], input_shape=[], dtype=tf.string, trainable=True)

In [ ]:
#Binary classifier for isExtra
model = Sequential()
model.add(hub_layer)
model.add(Dense(16, activation='relu', kernel_initializer=HeNormal()))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(input, outIsExtra, epochs=100, validation_split=0.2, batch_size=16, callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

Epoch 1/100
160/160 [==============================] - 12s 78ms/step - loss: 0.5580 - accuracy: 0.7090 - val_loss: 0.4320 - val_accuracy: 0.8250
Epoch 2/100
160/160 [==============================] - 12s 76ms/step - loss: 0.2642 - accuracy: 0.9105 - val_loss: 0.3736 - val_accuracy: 0.8344
Epoch 3/100
160/160 [==============================] - 12s 74ms/step - loss: 0.1137 - accuracy: 0.9680 - val_loss: 0.4281 - val_accuracy: 0.8234
Epoch 4/100
160/160 [==============================] - 12s 74ms/step - loss: 0.0484 - accuracy: 0.9914 - val_loss: 0.4880 - val_accuracy: 0.8313
Epoch 5/100
160/160 [==============================] - 12s 74ms/step - loss: 0.0297 - accuracy: 0.9937 - val_loss: 0.5290 - val_accuracy: 0.8250
Epoch 6/100
160/160 [==============================] - 12s 74ms/step - loss: 0.0241 - accuracy: 0.9945 - val_loss: 0.5750 - val_accuracy: 0.8234
Epoch 7/100
160/160 [==============================] - 12s 75ms/step - loss: 0.0213 - accuracy: 0.9945 - val_loss: 0.5864 - val_ac

In [ ]:
#Regression for the outChapter prediction
model2 = Sequential()
model2.add(hub_layer)
model2.add(Dense(64, activation='relu', kernel_initializer=HeNormal()))
model2.add(Dropout(0.3))
model2.add(Dense(32, activation='relu', kernel_initializer=HeNormal()))
model2.add(Dropout(0.3))
model2.add(Dense(16, activation='relu', kernel_initializer=HeNormal()))
model2.add(Dense(1))

model2.compile(optimizer='adam', loss='MAE', metrics=['MSE'])
model2.fit(input, outChapter, epochs=100, validation_split=0.2, batch_size=16, callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

Epoch 1/100
160/160 [==============================] - 12s 76ms/step - loss: 7.7642 - MSE: 95.6934 - val_loss: 6.4554 - val_MSE: 65.8331
Epoch 2/100
160/160 [==============================] - 12s 75ms/step - loss: 6.0233 - MSE: 60.1859 - val_loss: 5.6485 - val_MSE: 52.6971
Epoch 3/100
160/160 [==============================] - 12s 75ms/step - loss: 4.6804 - MSE: 39.0334 - val_loss: 5.2802 - val_MSE: 49.9920
Epoch 4/100
160/160 [==============================] - 12s 75ms/step - loss: 4.0322 - MSE: 30.0194 - val_loss: 5.2664 - val_MSE: 50.7780
Epoch 5/100
160/160 [==============================] - 12s 73ms/step - loss: 3.5300 - MSE: 23.8127 - val_loss: 5.3686 - val_MSE: 52.4415
Epoch 6/100
160/160 [==============================] - 12s 73ms/step - loss: 3.2067 - MSE: 19.4465 - val_loss: 5.4508 - val_MSE: 55.0905
Epoch 7/100
160/160 [==============================] - 12s 74ms/step - loss: 3.0576 - MSE: 18.6019 - val_loss: 5.5315 - val_MSE: 56.1143
Epoch 8/100
160/160 [====================

In [ ]:
#Regression for sectionFloat prediction
model3 = Sequential()
model3.add(hub_layer)
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.3))
model3.add(Dense(32, activation='relu'))
model3.add(Dropout(0.3))
model3.add(Dense(16, activation='relu'))
model3.add(Dense(1))

model3.compile(optimizer='adam', loss='MAE', metrics=['MSE'])
model3.fit(input, outSectionFloat, epochs=100, validation_split=0.2, batch_size=64, callbacks=[EarlyStopping(patience=5, restore_best_weights=True)])

Epoch 1/100
40/40 [==============================] - 3s 82ms/step - loss: 0.3599 - MSE: 0.2018 - val_loss: 0.2999 - val_MSE: 0.1320
Epoch 2/100
40/40 [==============================] - 3s 82ms/step - loss: 0.2846 - MSE: 0.1230 - val_loss: 0.2818 - val_MSE: 0.1194
Epoch 3/100
40/40 [==============================] - 3s 80ms/step - loss: 0.2617 - MSE: 0.1065 - val_loss: 0.2780 - val_MSE: 0.1171
Epoch 4/100
40/40 [==============================] - 3s 79ms/step - loss: 0.2359 - MSE: 0.0871 - val_loss: 0.2646 - val_MSE: 0.1084
Epoch 5/100
40/40 [==============================] - 3s 79ms/step - loss: 0.2051 - MSE: 0.0685 - val_loss: 0.2456 - val_MSE: 0.0930
Epoch 6/100
40/40 [==============================] - 3s 80ms/step - loss: 0.1868 - MSE: 0.0579 - val_loss: 0.2395 - val_MSE: 0.0896
Epoch 7/100
40/40 [==============================] - 3s 74ms/step - loss: 0.1614 - MSE: 0.0454 - val_loss: 0.2412 - val_MSE: 0.0915
Epoch 8/100
40/40 [==============================] - 3s 73ms/step - loss: 0.

In [ ]:
#Download test data
!wget http://phoenix.mit.bme.hu:49080/kgt/MIGT_2020_validation_V1.tsv

--2020-11-19 19:39:42--  http://phoenix.mit.bme.hu:49080/kgt/MIGT_2020_validation_V1.tsv
Resolving phoenix.mit.bme.hu (phoenix.mit.bme.hu)... 152.66.253.41
Connecting to phoenix.mit.bme.hu (phoenix.mit.bme.hu)|152.66.253.41|:49080... connected.
HTTP request sent, awaiting response... 200 OK
Length: 706809 (690K) [text/tab-separated-values]
Saving to: ‘MIGT_2020_validation_V1.tsv.3’

MIGT_2020_validatio 100%[===================>] 690.24K   751KB/s    in 0.9s    

2020-11-19 19:39:44 (751 KB/s) - ‘MIGT_2020_validation_V1.tsv.3’ saved [706809/706809]



In [ ]:
#Read test data
val = pd.read_csv("MIGT_2020_validation_V1.tsv", delimiter='\t')
npval=val.to_numpy()
npval=npval.reshape(1000)

In [ ]:
#Making the predictions, and some postprocessing
predsChapter = np.round(model2.predict(npval))
predsChapter[predsChapter > 27.0] = 27.0 
predsPosition = model3.predict(npval)
predsPosition[predsPosition > 1.0] = 1.0 
predsAIMA = np.round(model.predict(npval))

In [ ]:
#Megoldás összerakása és lementése
npres=np.concatenate((predsChapter, predsPosition, predsAIMA, npval.reshape(1000, 1)), axis=1)
np.savetxt("RESULTS.tsv", npres, delimiter="\t", fmt="%s")